In [ ]:
import pandas as pd
import numpy as np

In [ ]:
youtube_history = pd.read_csv("youtube_history_raw.csv")

In [ ]:
youtube_history.sample(2)

,source,name_of_video,url_of_video,name_of_channel,url_of_channel,time_stamp,date,month,day,year,hour,minute,second,AM/PM,time_zone
9705,YouTube,Shawn Mendes - It'll Be Okay (Lyric Video),https://www.youtube.com/watch?v=q_HNcBjHlPY,ShawnMendesVEVO,https://www.youtube.com/channel/UC4-TgOSMJHn-L...,Dec 4 2021 4:56:56 PM EST,4 Dec 2021,Dec,4,2021,4,56,56,PM,EST
13425,YouTube Music,exile,https://music.youtube.com/watch?v=Nm08yUg38tE,Taylor Swift - Topic,https://www.youtube.com/channel/UCPC0L1d253x-K...,Jul 8 2021 8:56:42 AM EST,8 Jul 2021,Jul,8,2021,8,56,42,AM,EST


# EST to IST
Considering I was in India before 8th August, 2023, I need to convert the EST time to IST time. Why? Because if I want to know at what time of the day do I consume content the most, I need to consider the time difference.

In [ ]:
from datetime import datetime
import pytz

In [ ]:
# Convert specific columns to int8
youtube_history['year'] = youtube_history['year'].astype('int16')
youtube_history['hour'] = youtube_history['hour'].astype('int8')
youtube_history['minute'] = youtube_history['minute'].astype('int8')
youtube_history['second'] = youtube_history['second'].astype('int8')

youtube_history['day'] = youtube_history['day'].astype(str).replace('[^0-9]', '', regex=True)
youtube_history['day'] = youtube_history['day'].astype('int8')

In [ ]:
def convert_est_to_ist(timestamp_str):
    # Define time zones
    est = pytz.timezone('US/Eastern')
    ist = pytz.timezone('Asia/Kolkata')

    # Convert string to datetime object
    # Reference: 'Aug 15 2021 8:58:43 AM EST'
    timestamp_str = timestamp_str.replace(' EST', '')
    timestamp = datetime.strptime(timestamp_str, '%b %d %Y %I:%M:%S %p')

    # Set time zone to EST
    timestamp = est.localize(timestamp)

    # Convert to UTC and then to IST
    timestamp_utc = timestamp.astimezone(pytz.utc)
    timestamp_ist = timestamp_utc.astimezone(ist)

    # Return formatted IST timestamp
    return timestamp_ist.strftime('%b %d %Y %I:%M:%S %p')

month_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
# print(month_dict["Aug"])

In [ ]:
# for index, entry in youtube_history.iterrows():
#   print(index)
#   print(entry)
#   print(youtube_history.at[index, "day"])
#   youtube_history.at[index, "day"] = np.random.randint(1, 31)
#   print(youtube_history.at[index, "day"])
#   break

In [ ]:
for index, entry in youtube_history.iterrows():
  if entry['month'] in ['Jan', 'Mar', 'May', 'Jul', 'Aug', 'Oct', 'Dec'] and entry['day'] > 31:
    changed_val = np.random.randint(1, 31)
    youtube_history.at[index, "day"] = changed_val
    entry['day'] = changed_val
  elif entry['month'] == 'Feb' and entry["day"] > 28:
    changed_val = np.random.randint(1, 28)
    youtube_history.at[index, "day"] = changed_val
    entry['day'] = changed_val
  elif entry["day"] > 30:
    changed_val = np.random.randint(1, 30)
    youtube_history.at[index, "day"] = changed_val
    entry['day'] = changed_val

  try:
    val = datetime(entry["year"], month_dict[entry["month"]], entry["day"]) < datetime(2023, month_dict["Aug"], 8)
  except:
    print(entry)

  if datetime(entry["year"], month_dict[entry["month"]], entry["day"]) < datetime(2023, month_dict["Aug"], 8):
    est_timestamp = f"{entry['month']} {entry['day']:02d} {entry['year']} {entry['hour']:02d}:{entry['minute']:02d}:{entry['second']:02d} {entry['AM/PM']} EST"
    try:
      temp = convert_est_to_ist(est_timestamp)
    except:
      est_timestamp.replace("  ", " ")
      print(est_timestamp)
      continue
    # Update entry with IST timestamp components
    ist_timestamp = convert_est_to_ist(est_timestamp)
    ist_timestamp = datetime.strptime(ist_timestamp, '%b %d %Y %I:%M:%S %p')
    youtube_history.at[index, 'month'] = ist_timestamp.strftime('%b')
    youtube_history.at[index, 'day'] = ist_timestamp.day
    youtube_history.at[index, 'year'] = ist_timestamp.year
    youtube_history.at[index, 'hour'] = ist_timestamp.hour
    youtube_history.at[index, 'minute'] = ist_timestamp.minute
    youtube_history.at[index, 'second'] = ist_timestamp.second
    youtube_history.at[index, 'AM/PM'] = ist_timestamp.strftime('%p')
    youtube_history.at[index, 'time_zone'] = 'IST'
    # print(f"Original EST Timestamp: {est_timestamp}, Converted IST Timestamp: {ist_timestamp}")
  else:
    continue
    # print("Timestamp after August 8, 2023. No conversion needed.")

Jul 07 2023 00:07:53 AM EST
Jun 11 2023 00:21:24 AM EST


# FILTERING TO GET ENTRIES OF 2023

In [ ]:
# Reference: selecting rows based on condition : rslt_df = dataframe[dataframe['Percentage'] > 70]
youtube_history_2023 = youtube_history[youtube_history['year'] == 2023]

In [ ]:
youtube_history_2023.sample(2)

,source,name_of_video,url_of_video,name_of_channel,url_of_channel,time_stamp,date,month,day,year,hour,minute,second,AM/PM,time_zone
21329,YouTube Music,Daydreamin' (Live from London),https://music.youtube.com/watch?v=Tb9G2YIx8w8,Ariana Grande - Topic,https://www.youtube.com/channel/UC0076UMUgEng8...,Sept 25 2023 01:29:41 AM EST,25 Sept 2023,Sep,25,2023,1,29,41,AM,EST
22204,YouTube,Taylor Swift - The Very First Night (Taylor's ...,https://www.youtube.com/watch?v=rVuyi-dPMIc,TaylorSwiftVEVO,https://www.youtube.com/channel/UCANLZYMidaCbL...,Apr 10 2023 09:11:08 AM EST,10 Apr 2023,Apr,10,2023,18,41,8,PM,IST


# Splitting into YT and YT-Music
I want to analyze both the videos and watch and the music I listen to. Video because it will be funa nd YT does not provide a YT wrapped. Music because I NEED AN ACCURATE WRAPPED FOR MYSELF BECAUSE I USED TWO SEPARATE ACCOUNTS IN 2023 😭

In [ ]:
channels_df = pd.DataFrame(columns=['name', 'url', 'freq']) # key = (url)
artists_df = pd.DataFrame(columns=['name', 'url', 'freq']) # key = (url)

videos_df = pd.DataFrame(columns=['name_vid', 'url_vid', 'duration', 'freq', 'channel_key']) # key = (url_vid)
# videos_df['freq'] = 0
# videos_df['duration'] = None
music_df = pd.DataFrame(columns=['name_song', 'url_song', 'duration', 'freq', 'artist_key']) # key = (url_song)
# music_df['freq'] = 0
# music_df['duration'] = None

cols = youtube_history.columns

videos_history = pd.DataFrame(columns = cols)
music_history = pd.DataFrame(columns = cols)
# videos_history = pd.DataFrame(columns = ['vid_df_key', 'time_stamp', 'date', 'day', 'month', 'year', 'hour', 'minute', 'second', 'AM/PM', 'time_zone']) # key = (vid_df_key, time_stamp)
# music_history = pd.DataFrame(columns = ['music_df_key', 'time_stamp', 'date', 'day', 'month', 'year', 'hour', 'minute', 'second', 'AM/PM', 'time_zone']) # key = (vid_df_key, time_stamp)

In [ ]:
# for index, entry in youtube_history_2023.iterrows():
#   print(index, entry)
#   break

In [ ]:
# Reference: df.loc[len(df.index)] = ['Amy', 89, 93]
# NOTE use of at : channels_df.at[channel_url, 'freq'] +=  1
for index, row in youtube_history_2023.iterrows():
  if row['source'] == 'YouTube':
    # IT IS A VIDEO
    # Step 01: Update channels_df
    channel_url = row['url_of_channel']
    if channel_url in channels_df['url']:
      channels_df.at[channel_url, 'freq'] +=  1 # NOTE
    else:
      channels_df.loc[channel_url] = [row['name_of_channel'], channel_url, 1] # NOTE
    # Step 02: Update videos_df
    video_url = row['url_of_video']
    if video_url in videos_df['url_vid']:
      videos_df.at[video_url, 'freq'] += 1
    else:
      videos_df.loc[video_url] = [row['name_of_video'], video_url, None, 1, channel_url]
    # Step 03: Update videos_history
    # ['vid_df_key', 'time_stamp', 'date', 'day', 'month', 'year', 'hour', 'minute', 'second', 'AM/PM', 'time_zone']
    videos_history.loc[len(videos_history)] = row
  else:
    # IT IS A SONG
    #STEP 01: Update artists_df
    artist_url = row['url_of_channel']
    if artist_url in artists_df['url']:
      # print(artists_df.at[artist_url, 'freq'])
      artists_df.at[artist_url, 'freq'] +=  1
    else:
      artists_df.loc[artist_url] = [row['name_of_channel'], artist_url, 1]
    # Step 02: Update music_df
    song_url = row['url_of_video']
    if song_url in music_df['url_song']:
      music_df.at[song_url, 'freq'] += 1
    else:
      music_df.loc[song_url] = [row['name_of_video'], song_url, None, 1, artist_url]
    # Step 03: Update music_history
    music_history.loc[len(music_history)] = row

In [ ]:
channels_df[channels_df['freq'] > 30]

,name,url,freq
https://www.youtube.com/channel/UCP0f9FXZ4g-MxAqSUreHpCw,Taylor Bell,https://www.youtube.com/channel/UCP0f9FXZ4g-Mx...,41
url_chan,name_chan,url_chan,65
https://www.youtube.com/channel/UC0rE2qq81of4fojo-KhO5rg,Tanmay Bhat,https://www.youtube.com/channel/UC0rE2qq81of4f...,54
https://www.youtube.com/channel/UCeYUKvRrRY9V3DYdZd1Z_eQ,Strolling The City,https://www.youtube.com/channel/UCeYUKvRrRY9V3...,50
https://www.youtube.com/channel/UCBkcw8h7epT_bK0QzuY2Bmg,ActionKid,https://www.youtube.com/channel/UCBkcw8h7epT_b...,55
https://www.youtube.com/channel/UCANLZYMidaCbLQFWXBC95Jg,TaylorSwiftVEVO,https://www.youtube.com/channel/UCANLZYMidaCbL...,34


In [ ]:
artists_df[artists_df['freq'] > 90].sort_values(by =  'freq', ascending=False)

,name,url,freq
https://www.youtube.com/channel/UCPC0L1d253x-KuMNwa05TpA,Taylor Swift - Topic,https://www.youtube.com/channel/UCPC0L1d253x-K...,989
https://www.youtube.com/channel/UCVacQ2t5GUZ2t_J3Ia9BynA,Harry Styles - Topic,https://www.youtube.com/channel/UCVacQ2t5GUZ2t...,218
https://www.youtube.com/channel/UCO_sphdxl8_K6mfXzzmkDGQ,Prateek Kuhad - Topic,https://www.youtube.com/channel/UCO_sphdxl8_K6...,131
https://www.youtube.com/channel/UCpe3TthvXb8kmRiFbkeDGCQ,Louis Tomlinson - Topic,https://www.youtube.com/channel/UCpe3TthvXb8km...,127
https://www.youtube.com/channel/UCE5XNpliPM-SmyFEp61tL_g,Olivia Rodrigo - Topic,https://www.youtube.com/channel/UCE5XNpliPM-Sm...,114
https://www.youtube.com/channel/UCwGXlFP4Ba5do7KoxRJYgVQ,Noah Kahan - Topic,https://www.youtube.com/channel/UCwGXlFP4Ba5do...,106
https://www.youtube.com/channel/UCCj0RlDqqahEB5BXVtDcPqg,5 Seconds of Summer - Topic,https://www.youtube.com/channel/UCCj0RlDqqahEB...,105
https://www.youtube.com/channel/UCDxKh1gFWeYsqePvgVzmPoQ,Arijit Singh - Topic,https://www.youtube.com/channel/UCDxKh1gFWeYsq...,101
https://www.youtube.com/channel/UCREhun2tPd7aQnJczZM_Mww,One Direction - Topic,https://www.youtube.com/channel/UCREhun2tPd7aQ...,96


In [ ]:
videos_df[videos_df['freq'] > 4]

,name_vid,url_vid,duration,freq,channel_key
url_vid,name_vid,url_vid,None,7,url_chan
https://www.youtube.com/watch?v=L-vptrNAhSs,1B,https://www.youtube.com/watch?v=L-vptrNAhSs,None,5,https://www.youtube.com/channel/UChlMfk7NmCl4J...
https://www.youtube.com/watch?v=sjc34vNBNhQ,I Watched all 236 Episodes of Friends,https://www.youtube.com/watch?v=sjc34vNBNhQ,None,6,https://www.youtube.com/channel/UCF_votze88WRD...
https://www.youtube.com/watch?v=WEDIj9JBTC8,William Ackman: Everything You Need to Know Ab...,https://www.youtube.com/watch?v=WEDIj9JBTC8,None,10,https://www.youtube.com/channel/UCvQECJukTDE2i...
https://www.youtube.com/watch?v=sPU0a9zoMco,Louis Tomlinson - Holding On To Heartache (Off...,https://www.youtube.com/watch?v=sPU0a9zoMco,None,6,https://www.youtube.com/channel/UCBSxVE6JoMg0W...
https://www.youtube.com/watch?v=qLCLvzTGFVM,The Local Train - Dil Mere (Official),https://www.youtube.com/watch?v=qLCLvzTGFVM,None,5,https://www.youtube.com/channel/UCTzjnbxgPIHYD...


In [ ]:
music_df[music_df['freq']>20].sort_values(by='freq', ascending=False)

,name_song,url_song,duration,freq,artist_key
https://music.youtube.com/watch?v=4UAZkexQ7Cg,Little Freak,https://music.youtube.com/watch?v=4UAZkexQ7Cg,None,29,https://www.youtube.com/channel/UCVacQ2t5GUZ2t...
https://music.youtube.com/watch?v=3lB7Oh076-8,champagne problems,https://music.youtube.com/watch?v=3lB7Oh076-8,None,26,https://www.youtube.com/channel/UCPC0L1d253x-K...
https://music.youtube.com/watch?v=FfBBGDZ4lzk,Satellite,https://music.youtube.com/watch?v=FfBBGDZ4lzk,None,25,https://www.youtube.com/channel/UCVacQ2t5GUZ2t...
https://music.youtube.com/watch?v=bqJ9I-3MG1g,Cornelia Street,https://music.youtube.com/watch?v=bqJ9I-3MG1g,None,24,https://www.youtube.com/channel/UCPC0L1d253x-K...
https://music.youtube.com/watch?v=Dfiw7EmRTC4,Paris,https://music.youtube.com/watch?v=Dfiw7EmRTC4,None,21,https://www.youtube.com/channel/UCQgUHOPJJrmzC...
https://music.youtube.com/watch?v=BnYokCSJDfw,ADHOORE,https://music.youtube.com/watch?v=BnYokCSJDfw,None,21,https://www.youtube.com/channel/UCe223VU7Yb2pR...
https://music.youtube.com/watch?v=Or58KytwxrI,Keep Driving,https://music.youtube.com/watch?v=Or58KytwxrI,None,21,https://www.youtube.com/channel/UCVacQ2t5GUZ2t...


# Extracting lengths of videos/songs using API
To find out how many hours/days I have spent consuming content this year, I'll use the YouTube Data API and extract lengths of songs and videos

In [ ]:
!pip install google-api-python-client
import re
import requests
from googleapiclient.discovery import build

In [ ]:
def extract_video_id(url):
    # Extract video ID from YouTube URL
    # pattern = r"(?<=youtu.be\/|watch\?v=|embed\/|youtube.com\/v\/|youtube.com\/embed\/|youtube.com\/watch\?v=|youtube.com\/watch\?feature=player_embedded&v=|%2Fvideos%2F|youtu.be%2F|y2u.be%2F|youtube.com%2Fuser%2F[^#]*%2Fvideos%2F|.*[&|\?]?v=|.*[\/\?&]v=|.*youtu.be\/|.*youtube.com\/embed\/|.*youtube.com\/v\/|.*youtube.com\/user\/[^#]*\/videos\/|.*youtube.com\/y\/[^#]*\/|.*youtube-nocookie.com\/v\/|.*youtube.googleapis.com\/v\/|.*youtube.com\/s[^#]*\/*\/([^\/\&\?]*))"
    # match = re.search(pattern, url)
    vid_id = url.split("=")[-1]
    if vid_id:
        # return match.group(1)
        # print(vid_id)
        return vid_id
    else:
        return None

def get_youtube_video_info(video_id, api_key):
    # Make a request to YouTube Data API to get video details
    api_service_name = "youtube"
    api_version = "v3"
    youtube = build(api_service_name, api_version, developerKey=api_key)

    request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id=video_id
    )
    response = request.execute()

    # Extract relevant information from the API response
    if "items" in response and len(response["items"]) > 0:
        video_info = response["items"][0]
        return video_info
    else:
        return None

def parse_duration(duration):
    # Remove the 'PT' at the beginning
    duration = duration[2:]

    # Initialize variables
    hours, minutes, seconds = 0, 0, 0

    # Parse hours if present
    if 'H' in duration:
        hours, duration = duration.split('H')
        hours = int(hours)

    # Parse minutes if present
    if 'M' in duration:
        minutes, duration = duration.split('M')
        minutes = int(minutes)

    # Parse seconds if present
    if 'S' in duration:
        seconds = int(duration.rstrip('S'))

    # Converting duration to minutes
    duration = hours*60 + minutes + seconds/60
    return duration

# Retrieved API Key using GCP
api_key = "YOUR_API_KEY"

In [ ]:
for index, row in videos_df.iterrows():
  video_url = row['url_vid']
  video_id = extract_video_id(video_url)
  if video_id:
    # Get video information using the API key and video ID
    video_info = get_youtube_video_info(video_id, api_key)
    if video_info:
        duration_raw = video_info['contentDetails']['duration']
        duration = parse_duration(duration_raw)
        videos_df.at[video_url, 'duration'] = duration
    else:
        videos_df.at[video_url, 'duration'] = 11.7 # Average length of a Youtube video
  else:
    videos_df.at[video_url, 'duration'] = 11.7 # Average length of a Youtube video

In [ ]:
indexDuration = videos_df[ videos_df['duration'] >= 1000 ].index
videos_df.drop(indexDuration, inplace=True)
videos_df[videos_df['duration'] > 120].sort_values(by = 'duration', ascending = False)

,name_vid,url_vid,duration,freq,channel_key
https://www.youtube.com/watch?v=pkYVOmU3MgA,Data Structures and Algorithms in Python - Ful...,https://www.youtube.com/watch?v=pkYVOmU3MgA,750.833333,3,https://www.youtube.com/channel/UC8butISFwT-Wl...
https://www.youtube.com/watch?v=LHBE6Q9XlzI,Python for Data Science - Course for Beginners...,https://www.youtube.com/watch?v=LHBE6Q9XlzI,739.866667,2,https://www.youtube.com/channel/UC8butISFwT-Wl...
https://www.youtube.com/watch?v=aLRV8ND5oLw,NEW YORK 🗽 Iconic Soho and Washington Square Park,https://www.youtube.com/watch?v=aLRV8ND5oLw,719.016667,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...
https://www.youtube.com/watch?v=saIr45wvt4M,4K NEW YORK Walking Tour 🗽NIGHT Walk BROOKLYN ...,https://www.youtube.com/watch?v=saIr45wvt4M,714.983333,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...
https://www.youtube.com/watch?v=jYHWCbevXkE,New York City NIGHT Walk 🗽DOWNTOWN MANHATTAN W...,https://www.youtube.com/watch?v=jYHWCbevXkE,714.983333,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...
https://www.youtube.com/watch?v=4n4kfYRJ9ss,New Year's Eve in NEW YORK CITY ☔ RAINY Walki...,https://www.youtube.com/watch?v=4n4kfYRJ9ss,714.966667,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...
https://www.youtube.com/watch?v=oRIzCoj5YzA,NEW YORK Walk in Midtown Manhattan 4K,https://www.youtube.com/watch?v=oRIzCoj5YzA,714.966667,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...
https://www.youtube.com/watch?v=Kxi3M1Qz2Gg,"NEW YORK 2023 Manhattanhenge Madness, NYC Walk...",https://www.youtube.com/watch?v=Kxi3M1Qz2Gg,714.933333,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...
https://www.youtube.com/watch?v=iLE8tW9AqR0,New York RAINY Lower Manhattan and SoHo Walk ☔...,https://www.youtube.com/watch?v=iLE8tW9AqR0,714.933333,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...
https://www.youtube.com/watch?v=LOSaABLSlhE,NEW YORK Walking tour - Sunset walk in Manhatt...,https://www.youtube.com/watch?v=LOSaABLSlhE,714.933333,1,https://www.youtube.com/channel/UCnvLGayHxWh5Y...


In [ ]:
for index, row in music_df.iterrows():
  video_url = row['url_song']
  video_id = extract_video_id(video_url)
  if video_id:
    # Get video information using the API key and video ID
    video_info = get_youtube_video_info(video_id, api_key)
    if video_info:
        duration_raw = video_info['contentDetails']['duration']
        duration = parse_duration(duration_raw)
        music_df.at[video_url, 'duration'] = duration
    else:
        music_df.at[video_url, 'duration'] = 11.7 # Average length of a Youtube video
  else:
    music_df.at[video_url, 'duration'] = 11.7 # Average length of a Youtube video

In [ ]:
music_df[(music_df['duration'] <= 11) & (music_df['freq'] > 1)].sort_values(by = 'duration', ascending=False).head(20)

,name_song,url_song,duration,freq,artist_key
https://music.youtube.com/watch?v=7SqC7_f_-0M,All Too Well (10 Minute Version) (Taylor's Ver...,https://music.youtube.com/watch?v=7SqC7_f_-0M,10.216667,11,https://www.youtube.com/channel/UCPC0L1d253x-K...
https://music.youtube.com/watch?v=aPnqx56V8-0,All Too Well (10 Minute Version) (Taylor's Ver...,https://music.youtube.com/watch?v=aPnqx56V8-0,10.216667,4,https://www.youtube.com/channel/UCPC0L1d253x-K...
https://music.youtube.com/watch?v=KC0AG8HfAi4,All Too Well (Sad Girl Autumn Version) - Recor...,https://music.youtube.com/watch?v=KC0AG8HfAi4,9.983333,11,https://www.youtube.com/channel/UCPC0L1d253x-K...
https://music.youtube.com/watch?v=GipThm50RkU,"Star Wars, Episode IV ""A New Hope"": Throne Roo...",https://music.youtube.com/watch?v=GipThm50RkU,7.966667,2,https://www.youtube.com/channel/UCfv8TEQ44ej9u...
https://music.youtube.com/watch?v=_fmA1RoHbzA,Kun Faya Kun,https://music.youtube.com/watch?v=_fmA1RoHbzA,7.85,4,https://www.youtube.com/channel/UCtJe0RYzgPddQ...
https://music.youtube.com/watch?v=vTBbi3qIC0s,Yun Hi Chala Chal,https://music.youtube.com/watch?v=vTBbi3qIC0s,7.45,2,https://www.youtube.com/channel/UCNeP2HGGg3xqu...
https://music.youtube.com/watch?v=aV5f5S5U4Sw,Beautiful Jesus,https://music.youtube.com/watch?v=aV5f5S5U4Sw,6.533333,2,https://www.youtube.com/channel/UCKOCtwOBPvPoY...
https://music.youtube.com/watch?v=3Cu5HLA_9T4,Dance for You,https://music.youtube.com/watch?v=3Cu5HLA_9T4,6.3,2,https://www.youtube.com/channel/UCoPQ_TWm8JZ5n...
https://music.youtube.com/watch?v=BViDG6QDe1w,Fine Line,https://music.youtube.com/watch?v=BViDG6QDe1w,6.3,3,https://www.youtube.com/channel/UCVacQ2t5GUZ2t...
https://music.youtube.com/watch?v=ZvOrqlB1iLI,Romeo And Juliet,https://music.youtube.com/watch?v=ZvOrqlB1iLI,6.016667,3,https://www.youtube.com/channel/UCA37MBUzQgLbU...


### Merging frequencies for duplicate song entries

In [ ]:
final_music_df = music_df.groupby('name_song').agg({'freq': 'sum', 'artist_key': 'first','name_song' : 'first', 'url_song': 'first', 'duration': 'first'})

In [ ]:
len(final_music_df)
len(final_music_df['name_song'].unique())

2067

In [ ]:
final_music_df.sort_values(by = 'freq', ascending=False).head(5)

,freq,artist_key,name_song,url_song,duration
name_song,,,,,
champagne problems,34,https://www.youtube.com/channel/UCPC0L1d253x-K...,champagne problems,https://music.youtube.com/watch?v=g9aNdMLYUrk,4.066667
Little Freak,29,https://www.youtube.com/channel/UCVacQ2t5GUZ2t...,Little Freak,https://music.youtube.com/watch?v=4UAZkexQ7Cg,3.383333
Daylight,27,https://www.youtube.com/channel/UCPC0L1d253x-K...,Daylight,https://music.youtube.com/watch?v=ppqxBjkziqo,4.9
Paris,26,https://www.youtube.com/channel/UCQgUHOPJJrmzC...,Paris,https://music.youtube.com/watch?v=Dfiw7EmRTC4,3.7
Satellite,26,https://www.youtube.com/channel/UCVacQ2t5GUZ2t...,Satellite,https://music.youtube.com/watch?v=FfBBGDZ4lzk,3.65


# Exporting data frames to csv files

In [ ]:
channels_df.to_csv("files/YTchannels.csv")
artists_df.to_csv("files/MusicArtists.csv")
videos_df.to_csv("files/YTvideos.csv")
final_music_df.to_csv("files/Songs.csv")
videos_history.to_csv("files/videos_history.csv")
music_history.to_csv("files/music_history.csv")
youtube_history_2023.to_csv("files/youtube_history_2023.csv")

# REFERENCE CODE
---

In [ ]:
# Extract video ID from the YouTube URL
# youtube_url = 'https://www.youtube.com/watch?v=WEDIj9JBTC8'
# youtube_url = 'https://music.youtube.com/watch?v=7SqC7_f_-0M'
# youtube_url = 'https://music.youtube.com/watch?v=u7hfwX81bY4'
video_id = extract_video_id(youtube_url)

if video_id:
    # Get video information using the API key and video ID
    video_info = get_youtube_video_info(video_id, api_key)

    if video_info:
        # Print relevant information
        for key, val in video_info.items():
          print(key, val)
        # print(f"Title: {video_info['snippet']['title']}")
        # print(f"Duration: {video_info['contentDetails']['duration']}")
        # print(f"Views: {video_info['statistics']['viewCount']}")
    else:
        print("Unable to fetch video information.")
else:
    print("Invalid YouTube URL.")

kind youtube#video
etag 7pciQGowdVjJNqCtYSC7jzSiG_8
id WEDIj9JBTC8
snippet {'publishedAt': '2012-11-27T19:42:33Z', 'channelId': 'UCvQECJukTDE2i6aCoMnS-Vg', 'title': 'William Ackman: Everything You Need to Know About Finance and Investing in Under an Hour | Big Think', 'description': 'Everything You Need to Know About Finance and Investing in Under an Hour\nWatch the newest video from Big Think: https://bigth.ink/NewVideo\nJoin Big Think Edge for exclusive videos: https://bigth.ink/Edge\n---------------------------------------------------------------------------------- \nBill Ackman is one of the top investors in the world, and he\'s said that he\'s aiming to have "one of the greatest investment track records of all time."  As the CEO of Pershing Square Capital Management, the hedge fund he founded, he oversees $19 billion in assets.\n\nBut before he became one of the elite, he learned the basics of investing in his early 20s.\n\nThis Big Think video is aimed at young professionals just

In [ ]:
!pip install youtube-search-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
from youtubesearchpython import VideosSearch

def get_album_name(song_name):
    try:
        # Searching for the song on YouTube
        videos_search = VideosSearch(song_name, limit = 1)
        results = videos_search.result()

        # Extracting album name from the search results
        video_info = results['result'][0]
        album_name = video_info.get('album', 'Album information not available')

        return album_name

    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
song_names = ["As It Was", "Daylight", "Paris"]
for song_name in song_names:
    album_name = get_album_name(song_name)
    print(f"Song: {song_name}, Album: {album_name}")


Song: As It Was, Album: Album information not available
Song: Daylight, Album: Album information not available
Song: Paris, Album: Album information not available


In [ ]:
# Drop rows with certain condition
indexAge = df[ (df['Age'] >= 20) & (df['Age'] <= 25) ].index
df.drop(indexAge , inplace=True)

In [ ]:
from datetime import datetime
import pytz

# Sample data with timestamp, day, month, year, hour, minute, second, AM/PM columns
data = [
    {"timestamp": "2023-08-01 12:30:45 PM", "day": 1, "month": 'Aug', "year": 2023, "hour": 12, "minute": 30, "second": 45, "am_pm": "PM"},
    # Add more rows with your data
]

def convert_est_to_ist(timestamp_str):
    # Define time zones
    est = pytz.timezone('US/Eastern')
    ist = pytz.timezone('Asia/Kolkata')

    # Convert string to datetime object
    timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d %I:%M:%S %p')

    # Set time zone to EST
    timestamp = est.localize(timestamp)

    # Convert to IST
    timestamp_ist = timestamp.astimezone(ist)

    # Return formatted IST timestamp
    return timestamp_ist.strftime('%Y-%m-%d %I:%M:%S %p')

month_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
print(month_dict["Aug"])

# Example usage
for entry in data:
    if datetime(entry["year"], month_dict[entry["month"]], entry["day"]) < datetime(2023, month_dict["Aug"], 8):
        est_timestamp = f"{entry['month']} {entry['day']:02d} {entry['year']} {entry['hour']:02d}:{entry['minute']:02d}:{entry['second']:02d} {entry['am_pm']} EST"
        ist_timestamp = convert_est_to_ist(est_timestamp)
        print(f"Original EST Timestamp: {est_timestamp}, Converted IST Timestamp: {ist_timestamp}")
    else:
        print("Timestamp after August 8, 2023. No conversion needed.")
